In [527]:
include("utils.jl")

const kline_limit = 100 # number of candlesticks (klines) to fetch and show

coin_pair = Observable("ETCUSDT")
kline_interval = Observable("1m")
leverage = Observable(50)
stop_loss_pct = Observable(3e-3)
take_profit_pct = Observable(5e-3);

In [656]:
function get_linesegs(Low, High, kline_limit)
    linesegs = []
    for i in 1:kline_limit
        push!(linesegs, Point2f(i, Low[i]))
        push!(linesegs, Point2f(i, High[i]))
    end
    linesegs = Point2f.(linesegs)
end

function get_ohlc(symbol, interval, limit)
    klines = get_klines(symbol, interval=interval, limit=limit)
    Open = parse.(Float64, [k[2] for k in klines])
    High = parse.(Float64, [k[3] for k in klines])
    Low = parse.(Float64, [k[4] for k in klines])
    Close = parse.(Float64, [k[5] for k in klines])
    colors = Close .>= Open

    linesegs = get_linesegs(Low, High, limit)
    last_price = Close[end]
    last_color = colors[end] ? :green : :red
    y_min = minimum(Low) * 0.997
    y_max = maximum(High) * 1.003

    Open, High, Low, Close, colors, linesegs, last_price, last_color, y_min, y_max
end

ohlcs = @lift get_ohlc($coin_pair, $kline_interval, kline_limit)

Open = @lift $ohlcs[1]
High = @lift $ohlcs[2]
Low = @lift $ohlcs[3]
Close = @lift $ohlcs[4]
colors = @lift $ohlcs[5]
linesegs = @lift $ohlcs[6]
last_price = @lift $ohlcs[7]
last_color = @lift $ohlcs[8]
y_min = @lift $ohlcs[9]
y_max = @lift $ohlcs[10];

ACCI = @lift acci($Close);

# true_leverage = @lift change_leverage(user, $coin_pair, $leverage);

In [658]:
true_leverage

Observable{Int64} with 0 listeners. Value:
20

In [664]:
fig = Figure(font="sans", fontsize=20)
vratio = 4
hratio = 4

ax1 = Axis(fig[1:vratio, 1:hratio], title=coin_pair, yaxisposition=:right)
ax2 = Axis(fig[vratio+1, 1:hratio], title="CCI", yaxisposition=:right)
deactivate_interaction!(ax1, :rectanglezoom)
deactivate_interaction!(ax2, :rectanglezoom)


barplot!(ax1, 1:kline_limit, Open, fillto=Close,
    color=colors, strokewidth=0.5, strokecolor=colors, colormap=cmap)
linesegments!(ax1, linesegs, color=colors, colormap=cmap)
hlines!(ax1, last_price, color=last_color, linestyle=:dash, linewidth=1)

# choose Long or Short mode
mode = Observable("Long")
text_mode = @lift $mode * " mode"
text_mode_color = @lift $mode == "Long" ? :green : :red
wallet_balance = Observable(get_balance(user, "USDT")[1])
text_balance = @lift "Blance = " * string($wallet_balance) * "USDT"

on(events(ax1).keyboardbutton) do event
    if ispressed(fig, Keyboard.s)
        mode[] = "Short"
    elseif ispressed(fig, Keyboard.l)
        mode[] = "Long"
    end
end

# show prices along with the mouse movements
mouse_pos = lift(events(ax1).mouseposition) do mp
    mouseposition(ax1.scene)
end

mouse_pos_x = @lift $mouse_pos[1]
mouse_open_price = @lift $mouse_pos[2]
mouse_stop_loss_price = @lift $mode == "Long" ? $mouse_open_price * (1 - $stop_loss_pct) : $mouse_open_price * (1 + $stop_loss_pct)
mouse_take_profit_price = @lift $mode == "Long" ? $mouse_open_price * (1 + $take_profit_pct) : $mouse_open_price * (1 - $take_profit_pct)

vlines!(ax1, mouse_pos_x, color=:red, linestyle=:dash, linewidth=1)
hlines!(ax1, mouse_open_price, linestyle=:dash, linewidth=1, color=:black)
hlines!(ax1, mouse_stop_loss_price, linestyle=:dash, linewidth=1, color=:red)
hlines!(ax1, mouse_take_profit_price, linestyle=:dash, linewidth=1, color=:green)

mouse_text_coor = 30
mouse_open_text = text!(ax1, mouse_text_coor, mouse_open_price, text=mode, color=:black)
mouse_stop_loss_text = text!(ax1, mouse_text_coor, mouse_stop_loss_price, text="Stop Loss", color=:red)
mouse_take_profit_text = text!(ax1, mouse_text_coor, mouse_take_profit_price, text="Take Profit", color=:green)

x_min = -3
x_max = 120
scatter!(ax1, mouse_pos; markersize=10, color=:red)
xlims!(ax1, x_min, x_max)
ylims!(ax1, y_min[], y_max[])

menu_symbol = Menu(fig, options=["BTCUSDT", "EOSUSDT", "ETHUSDT", "ETCUSDT"], default=coin_pair[])
menu_kline_interval = Menu(fig, options=["1m", "3m", "5m", "15m", "1h", "4h"], default=kline_interval[])
menu_leverage = Menu(fig, options=zip(["10", "20", "30", "50", "75", "100"], [10, 20, 30, 50, 75, 100]), default=string(leverage[]))
menu_stop_loss = Menu(fig, options=zip(["1", "2", "3", "4", "5", "10"], [1e-3, 2e-3, 3e-3, 4e-3, 5e-3, 1e-2]), default=string(Int(stop_loss_pct[] * 1000)))
menu_take_profit = Menu(fig, options=zip(["1", "2", "3", "4", "5", "10"], [1e-3, 2e-3, 3e-3, 4e-3, 5e-3, 1e-2]), default=string(Int(take_profit_pct[] * 1000)))


fig[1:vratio+1, hratio+1] = vgrid!(
    Label(fig, "Symbol", width=nothing), menu_symbol,
    Label(fig, "TimeInterval", width=nothing), menu_kline_interval,
    Label(fig, "Leverage", width=nothing), menu_leverage,
    Label(fig, "Stop Loss ‰", width=nothing), menu_stop_loss,
    Label(fig, "Take Profit ‰", width=nothing), menu_take_profit,
    Label(fig, text_mode, width=nothing, color=text_mode_color),
    Label(fig, text_balance, width=nothing),
    Label(fig, "UnP&L", width=nothing),
    Label(fig, "PAmount", width=nothing),
    Label(fig, "PPrice", width=nothing),
    Label(fig, "OpenOrder", width=nothing),
    Label(fig, "SLOrder", width=nothing),
    Label(fig, "TPOrder", width=nothing);
    tellheight=false)



on(menu_symbol.selection) do s
    coin_pair[] = s
    ylims!(ax1, y_min[], y_max[])
end

on(menu_leverage.selection) do s
    leverage[] = s
    change_leverage(user, coin_pair[], leverage[])
end

on(menu_stop_loss.selection) do s
    stop_loss_pct[] = s
end

on(menu_take_profit.selection) do s
    take_profit_pct[] = s
end

on(menu_kline_interval.selection) do s
    kline_interval[] = s
    ylims!(ax1, y_min[], y_max[])
end

do_long_or_short = "long"
open_price = stop_loss_price = take_profit_price = 0.0

is_open_order = false
text_coor = 102 # the x-coordinate where to put the text
sig_digits = 5 # number of significant digits

on(events(ax1).keyboardbutton) do event
    global is_open_order
    if ispressed(fig, Keyboard.r) & (!is_open_order)
        global open_price = mouse_open_price[]
        global stop_loss_price = mouse_stop_loss_price[]
        global take_profit_price = mouse_take_profit_price[]

        global open_line = hlines!(ax1, open_price, color=:blue, linewidth=1)
        global open_text = text!(ax1, text_coor, open_price, text="$(mode[])@$(round(open_price, sigdigits=sig_digits))")
        global stop_loss_line = hlines!(ax1, stop_loss_price, color=:red, linewidth=1)
        global stop_loss_text = text!(ax1, text_coor, stop_loss_price, text="SL@$(round(stop_loss_price, sigdigits=sig_digits))")
        global take_profit_line = hlines!(ax1, take_profit_price, color=:green, linewidth=1)
        global take_profit_text = text!(ax1, text_coor, take_profit_price, text="TP@$(round(take_profit_price, sigdigits=sig_digits))")
        global is_open_order = true
    end
end

on(events(ax1).keyboardbutton) do event
    if ispressed(fig, Keyboard.c)
        delete!(ax1, open_line)
        delete!(ax1, open_text)
        delete!(ax1, stop_loss_line)
        delete!(ax1, stop_loss_text)
        delete!(ax1, take_profit_line)
        delete!(ax1, take_profit_text)
        global is_open_order = false
    end
end

# CCI indicator
lines!(ax2, ACCI, color=:blue)
vlines!(ax2, mouse_pos_x, color=:red, linestyle=:dash, linewidth=1)
hlines!(ax2, 0, color=:black, linestyle=:dash, linewidth=0.5)
hlines!(ax2, -100, color=:red, linewidth=0.5)
hlines!(ax2, 100, color=:red, linewidth=0.5)
ylims!(ax2, -200, 200)
linkxaxes!(ax1, ax2)

# fig[4, 4] = buttongrid = GridLayout(tellwidth=false)


# buttonlabels = ["long", "short"]

# buttons = buttongrid[1:length(buttonlabels), 1] = [Button(fig, label=l, buttoncolor_active=:blue) for l in buttonlabels]
# button_long = Button(fig, label = "do long")
# button_short = Button(fig, label = "do short")

# is_long = Observable(false)
# on(is_long) do val # Create a listener
#     println("is_long has been changed to $val")
# end

# on(buttons[1].clicks) do clicks
#     is_long[] = !is_long[]
#     if is_long[]
#         hlines!(ax, [100], color=:blue)
#         buttons[1] = Button(fig, label="long2", color=:blue)
#     end
# end

# on(buttons[2].clicks) do clicks
#     notify("short")
# end

# run = Button(fig[4, 4]; label="run", tellwidth=false)
display(fig)

GLMakie.Screen(...)

In [666]:
tt = 1
while tt < 100
    coin_pair[] = coin_pair[]
    kline_interval[] = kline_interval[]
    ylims!(ax1, y_min[], y_max[])
    sleep(0.005)
    tt += 1
end

In [495]:
frames = 1:60
record(fig, "video.mp4", frames; framerate=20) do i # i = frame number
    coin_pair[] = coin_pair[]
    kline_interval[] = kline_interval[]
    ylims!(ax1, y_min[], y_max[])
    sleep(0.005)
    # any other manipulation of the figure here...
end # for each step of this loop, a frame is recorded


"video.mp4"

In [459]:
user = User("/home/czc/projects/working/stock/GridBot.jl/api_key_secrets/binance/test.json");

In [587]:
account_info = get_account_info(user)

Dict{String, Any} with 18 entries:
  "totalInitialMargin"          => "0.00000000"
  "totalPositionInitialMargin"  => "0.00000000"
  "updateTime"                  => 0
  "positions"                   => Any[Dict{String, Any}("askNotional"=>"0", "u…
  "totalCrossUnPnl"             => "0.00000000"
  "canDeposit"                  => true
  "feeTier"                     => 0
  "totalWalletBalance"          => "1.00000000"
  "canWithdraw"                 => true
  "totalOpenOrderInitialMargin" => "0.00000000"
  "totalMaintMargin"            => "0.00000000"
  "totalMarginBalance"          => "1.00000000"
  "totalCrossWalletBalance"     => "1.00000000"
  "availableBalance"            => "1.00000000"
  "maxWithdrawAmount"           => "1.00000000"
  "assets"                      => Any[Dict{String, Any}("marginAvailable"=>tru…
  "totalUnrealizedProfit"       => "0.00000000"
  "canTrade"                    => true

In [592]:
# get positions 
filter(x -> x["symbol"] == coin_pair[], account_info["positions"])[1]


Dict{String, Any} with 17 entries:
  "askNotional"            => "0"
  "updateTime"             => 1661150998013
  "maxNotional"            => "10000"
  "bidNotional"            => "0"
  "positionInitialMargin"  => "0"
  "openOrderInitialMargin" => "0"
  "isolated"               => true
  "notional"               => "0"
  "symbol"                 => "ETCUSDT"
  "positionAmt"            => "0.00"
  "positionSide"           => "BOTH"
  "unrealizedProfit"       => "0.00000000"
  "initialMargin"          => "0"
  "entryPrice"             => "0.0"
  "leverage"               => "75"
  "isolatedWallet"         => "0"
  "maintMargin"            => "0"

In [598]:
"""
Get open orders.
"""
function get_open_orders(user)
    query = query_head()
    signature = do_sign(query, user)
    body = "$BINANCE_FUTURE_BASE_URL/fapi/v1/openOrders?$query&signature=$signature"
    response = HTTP.request("GET", body, user.headers)
    response_to_json(response.body)
end


get_open_orders

In [599]:
get_open_orders(user)

Any[]

In [793]:
"""
Create an order dict to be executed later.
"""
function create_order_dict(symbol::String, order_side::String;
    quantity::Float64=0.0, order_type::String="LIMIT",
    price::Float64=0.0, stop_price::Float64=0.0,
    iceberg_qty::Float64=0.0, new_client_order_id::String="")

    if quantity <= 0.0
        error("Quantity cannot be <=0 for order type.")
    end

    println("$order_side => $symbol qty: $quantity, price: $price ")

    order = Dict(
        "symbol" => symbol,
        "side" => order_side,
        "type" => order_type,
        "quantity" => Printf.@sprintf("%.8f", quantity),
        "newOrderRespType" => "FULL",
        "recvWindow" => 60000
    )

    if new_client_order_id != ""
        order["newClientOrderId"] = new_client_order_id
    end

    if order_type in ["LIMIT", order_type == "LIMIT_MAKER"]
        if price <= 0.0
            error("Price cannot be <= 0 for order type.")
        end

        if price * quantity < 5.0
            error("Order's notional must be no smaller than 5.0 (unless you choose reduce only)")
        end

        order["price"] = Printf.@sprintf("%.8f", price)
    end

    if order_type == "STOP_LOSS" || order_type == "TAKE_PROFIT"
        if stop_price <= 0.0
            error("Stop price cannot be <= 0 for order type.")
        end
        order["stopPrice"] = Printf.@sprintf("%.8f", stop_price)
    end

    if order_type in ["STOP_LOSS_LIMIT", "TAKE_PROFIT_LIMIT"]
        # if price <= 0.0 || stop_price <= 0.0
        #     error("Price or stop price cannot be <= 0 for order type.")
        # end
        order["price"] = Printf.@sprintf("%.8f", price)
        order["stopPrice"] = Printf.@sprintf("%.8f", stop_price)
    end

    if order_type == "TAKE_PROFIT"
        if price <= 0.0 || stop_price <= 0.0
            error("Price or stop price cannot be <= 0 for STOP_LOSS_LIMIT order type.")
        end
        order["price"] = Printf.@sprintf("%.8f", price)
        order["stopPrice"] = Printf.@sprintf("%.8f", stop_price)
    end

    if order_type in ["LIMIT", "STOP_LOSS_LIMIT", "TAKE_PROFIT_LIMIT"]
        order["timeInForce"] = "GTC"
    end

    order
end


create_order_dict

In [794]:
coin_pair[]

"ETCUSDT"

In [805]:
order0 = create_order_dict(coin_pair[], "BUY";
        quantity=0.2, order_type="LIMIT",
        price=32.0)

# order1 = create_order_dict(coin_pair[], "SELL";
#         quantity=string2float(get_position(user, coin_pair[])[1]["positionAmt"]), order_type="LIMIT",
#         price=32.699)

# order1["positionSide"] = "LONG"
# order1["stopPrice"] = 32.841
# order1["type"] = "LIMIT"
# order1["reduceOnly"] = "true"
# order1["closePosition"] = "true"
# order1

order0["newClientOrderId"] = "limit_long"
order0

BUY => ETCUSDT qty: 0.2, price: 32.0 


Dict{String, Any} with 9 entries:
  "price"            => "32.00000000"
  "timeInForce"      => "GTC"
  "side"             => "BUY"
  "newOrderRespType" => "FULL"
  "symbol"           => "ETCUSDT"
  "recvWindow"       => 60000
  "newClientOrderId" => "limit_long"
  "type"             => "LIMIT"
  "quantity"         => "0.20000000"

In [807]:
order = execute_order(order0, user)

Dict{String, Any} with 21 entries:
  "price"         => "32"
  "timeInForce"   => "GTC"
  "priceProtect"  => false
  "orderId"       => 13409708101
  "executedQty"   => "0"
  "updateTime"    => 1661695608410
  "clientOrderId" => "limit_long"
  "stopPrice"     => "0"
  "status"        => "NEW"
  "origType"      => "LIMIT"
  "avgPrice"      => "0.00000"
  "cumQty"        => "0"
  "cumQuote"      => "0"
  "reduceOnly"    => false
  "closePosition" => false
  "symbol"        => "ETCUSDT"
  "positionSide"  => "BOTH"
  "origQty"       => "0.20"
  "side"          => "BUY"
  ⋮               => ⋮

In [808]:
cancel_order(order, user)

Dict{String, Any} with 21 entries:
  "price"         => "32"
  "timeInForce"   => "GTC"
  "priceProtect"  => false
  "orderId"       => 13409708101
  "executedQty"   => "0"
  "updateTime"    => 1661695670113
  "clientOrderId" => "limit_long"
  "stopPrice"     => "0"
  "status"        => "CANCELED"
  "origType"      => "LIMIT"
  "avgPrice"      => "0.00000"
  "cumQty"        => "0"
  "cumQuote"      => "0"
  "reduceOnly"    => false
  "closePosition" => false
  "symbol"        => "ETCUSDT"
  "positionSide"  => "BOTH"
  "origQty"       => "0.20"
  "side"          => "BUY"
  ⋮               => ⋮

In [671]:
"""
Execute an order to the account.
"""
function execute_order(order::Dict, user::User)
    order_params = order_dict_to_params(order)
    query = "$order_params&timestamp=$(timestamp_now())"
    signature = do_sign(query, user)
    body = "$BINANCE_FUTURE_ORDER_URL?$query&signature=$signature"   
    response = HTTP.request("POST", body, user.headers)
    response_to_json(response.body)
end



execute_order

In [680]:

"""
Cancel an order from user.
"""
function cancel_order(order, user)
    query = "recvWindow=50000&timestamp=$(timestamp_now())&symbol=$(order["symbol"])&origClientOrderId=$(order["clientOrderId"])"
    signature = do_sign(query, user)
    body = "$BINANCE_FUTURE_ORDER_URL?$query&signature=$signature"
    response = HTTP.request("DELETE", body, user.headers)
    response_to_json(response.body)
end

cancel_order

In [613]:
change_multi_assets_mode(user, "single")
change_position_mode(user, "hedge")

In [622]:
# Get current position information.
function get_position(user, symbol)
    query = "$(query_head())&symbol=$symbol"
    signature = do_sign(query, user)
    body = "$BINANCE_FUTURE_BASE_URL/fapi/v2/positionRisk?$query&signature=$signature"
    response = HTTP.request("GET", body, user.headers)
    response_to_json(response.body)
end

function get_position(user)
    query = "$(query_head())"
    signature = do_sign(query, user)
    body = "$BINANCE_FUTURE_BASE_URL/fapi/v2/positionRisk?$query&signature=$signature"
    response = HTTP.request("GET", body, user.headers)
    response_to_json(response.body)
end

get_position (generic function with 2 methods)

In [624]:
get_position(user)[1]

Dict{String, Any} with 15 entries:
  "isolatedMargin"   => "0.00000000"
  "updateTime"       => 0
  "maxNotionalValue" => "25000"
  "liquidationPrice" => "0"
  "marginType"       => "cross"
  "notional"         => "0"
  "isAutoAddMargin"  => "false"
  "symbol"           => "RAYUSDT"
  "positionAmt"      => "0.0"
  "positionSide"     => "LONG"
  "markPrice"        => "0.00000000"
  "entryPrice"       => "0.0"
  "leverage"         => "20"
  "unRealizedProfit" => "0.00000000"
  "isolatedWallet"   => "0"

In [785]:
string2float(get_position(user, coin_pair[])[1]["positionAmt"])

0.94

In [647]:
change_leverage(user, coin_pair[], 75)
change_margin_mode(user, coin_pair[], "ISOLATED")
change_position_mode(user, "dual")

In [809]:

"""
Get Binance information.
"""
get_Binance_info() = request_get(BINANCE_FUTURE_EXCHANGEINFO_URL)


get_Binance_info

In [811]:
tt = get_Binance_info()

Dict{String, Any} with 7 entries:
  "symbols"         => Any[Dict{String, Any}("orderTypes"=>Any["LIMIT", "MARKET…
  "rateLimits"      => Any[Dict{String, Any}("intervalNum"=>1, "interval"=>"MIN…
  "exchangeFilters" => Any[]
  "serverTime"      => 1661695211722
  "timezone"        => "UTC"
  "futuresType"     => "U_MARGINED"
  "assets"          => Any[Dict{String, Any}("autoAssetExchange"=>"-10000", "as…

In [815]:
@show keys(tt["symbols"][1])

keys((tt["symbols"])[1]) = ["orderTypes", "timeInForce", "deliveryDate", "marginAsset", "triggerProtect", "contractType", "underlyingSubType", "underlyingType", "baseAssetPrecision", "quoteAsset", "pair", "status", "liquidationFee", "quantityPrecision", "pricePrecision", "symbol", "onboardDate", "marketTakeBound", "filters", "baseAsset", "settlePlan", "quotePrecision", "requiredMarginPercent", "maintMarginPercent"]


KeySet for a Dict{String, Any} with 24 entries. Keys:
  "orderTypes"
  "timeInForce"
  "deliveryDate"
  "marginAsset"
  "triggerProtect"
  "contractType"
  "underlyingSubType"
  "underlyingType"
  "baseAssetPrecision"
  "quoteAsset"
  "pair"
  "status"
  "liquidationFee"
  "quantityPrecision"
  "pricePrecision"
  "symbol"
  "onboardDate"
  "marketTakeBound"
  "filters"
  ⋮

In [816]:
tt["symbols"][1]

Dict{String, Any} with 24 entries:
  "orderTypes"         => Any["LIMIT", "MARKET", "STOP", "STOP_MARKET", "TAKE_P…
  "timeInForce"        => Any["GTC", "IOC", "FOK", "GTX"]
  "deliveryDate"       => 4133404800000
  "marginAsset"        => "USDT"
  "triggerProtect"     => "0.0500"
  "contractType"       => "PERPETUAL"
  "underlyingSubType"  => Any["PoW"]
  "underlyingType"     => "COIN"
  "baseAssetPrecision" => 8
  "quoteAsset"         => "USDT"
  "pair"               => "BTCUSDT"
  "status"             => "TRADING"
  "liquidationFee"     => "0.015000"
  "quantityPrecision"  => 3
  "pricePrecision"     => 2
  "symbol"             => "BTCUSDT"
  "onboardDate"        => 1569398400000
  "marketTakeBound"    => "0.05"
  "filters"            => Any[Dict{String, Any}("maxPrice"=>"4529764", "filterT…
  ⋮                    => ⋮